In [42]:
import pandas as pd
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout


In [43]:
import sys
sys.path.append("..")
from src.Eval import eval_llm

import os
import io
from contextlib import redirect_stdout

def clean_dict(d):
    """
    Recursively remove keys from a dictionary if they map to empty lists.
    """
    if isinstance(d, dict):
        return {k: clean_dict(v) for k, v in d.items() if clean_dict(v) != {} and v != []}
    elif isinstance(d, list):
        return [clean_dict(item) for item in d if clean_dict(item) != {} and item != []]
    else:
        return d

def parse_filename(filename):
    parts = filename.replace(".jsonl", "").split("_")
    dataset = parts[0]
    k_shot = parts[1].split(".")[0]
    k = int(k_shot.replace("shot", ""))
    if k and k != 0 and len(parts) == 3:
        sampling_method = parts[2]
    else:
        sampling_method = "rand"
    return dataset, k, sampling_method

def silent_eval_llm(fpath):
    with io.StringIO() as buf, redirect_stdout(buf):
        # Call the actual eval_llm function
        return eval_llm(fpath)
    
def aggregated_results_llm(
    results_dir,
    ent_types=["overall", "Artist", "WoA"],
    eval_schemas=["strict", "exact", "ent_type"],
    metrics=["f1", "f1_macro", "f1_micro", "precision", "precision_macro", "recall", "recall_macro", "missed", "spurious", "incorrect"],
    datasets=["dataset1", "dataset2", "dataset3", "dataset4"],
    sampling_methods=["rand", "tfidf", ""],
    ks=[0,5,15,25,35,45]
):
    results = {}
    for schema in eval_schemas:
        results[schema] = {}
        for ent_type in ent_types:
            results[schema][ent_type] = {}
            for metric in metrics:
                results[schema][ent_type][metric] = {}
                for model in [m for m in os.listdir(results_dir) if m != "archive"]:
                    results[schema][ent_type][metric][model] = {}
                    for sampling_method in sampling_methods:
                        results[schema][ent_type][metric][model][sampling_method] = {}
                        for k in ks:
                            results[schema][ent_type][metric][model][sampling_method][k] = []

    model_dirs = [
        os.path.join(results_dir, d)
        for d in os.listdir(results_dir)
        if os.path.isdir(os.path.join(results_dir, d)) and d != "archive"
    ]

    for model_dir in model_dirs:
        model = model_dir.split(os.sep)[-1]
        files = [
            os.path.join(model_dir, f)
            for f in os.listdir(model_dir)
            if os.path.isfile(os.path.join(model_dir, f))
        ]

        for fpath in files:
            print(fpath)
            filename = os.path.basename(fpath)
            dataset, k, sampling_method = parse_filename(filename)

            if dataset in datasets and k in ks:
                predictions = silent_eval_llm(fpath)

                for ent_type in ent_types:
                    for schema in eval_schemas:
                        for metric in metrics:
                            key = f"{ent_type}_{schema}_{metric}"
                            if key in predictions.keys():
                                results[schema][ent_type][metric][model][sampling_method][k].append(predictions[key])
    return clean_dict(results)

results = aggregated_results_llm("../output/reddit+shsyt/")
#results_tfidf = aggregated_results_llm("../output/tfidf_sampling")



2024-09-06 08:14:02 root INFO: Imported 660 predictions for 660 true examples


../output/reddit+shsyt_backup/llama3.1/dataset2_5shot_tfidf.jsonl


2024-09-06 08:14:02 root INFO: Imported 658 predictions for 658 true examples


../output/reddit+shsyt_backup/llama3.1/dataset4_35shot_tfidf.jsonl


2024-09-06 08:14:02 root INFO: Imported 660 predictions for 660 true examples


...

../output/reddit+shsyt_backup/llama3.1:70b/dataset4_5shot_rand.jsonl


2024-09-06 08:14:16 root INFO: Imported 658 predictions for 658 true examples


../output/reddit+shsyt_backup/llama3.1:70b/dataset3_35shot_rand.jsonl


2024-09-06 08:14:16 root INFO: Imported 660 predictions for 660 true examples


../output/reddit+shsyt_backup/llama3.1:70b/dataset2_25shot_tfidf.jsonl
../output/reddit+shsyt_backup/llama3.1-8b/dataset2_15shot3.jsonl
../output/reddit+shsyt_backup/llama3.1-8b/dataset2_15shot2.jsonl


In [44]:
import pandas as pd
import numpy as np

def results_to_dataframe(aggregated_results, agg_func='mean'):
    data = []

    for schema, schema_dict in aggregated_results.items():
        for ent_type, ent_type_dict in schema_dict.items():
            for metric, metric_dict in ent_type_dict.items():
                for model, model_dict in metric_dict.items():
                    for sampling, sampling_dict in model_dict.items():
                        for k_shot, values in sampling_dict.items():
                            if values:
                                if agg_func == 'mean':
                                    agg_value = np.mean(values)
                                elif agg_func == 'sum':
                                    agg_value = np.sum(values)
                                else:
                                    raise ValueError("Invalid aggregation function. Use 'mean' or 'sum'.")
                                
                                data.append([schema, ent_type, metric, model, sampling, k_shot, agg_value])

    df = pd.DataFrame(data, columns=['Schema', 'Entity Type', 'Metric', 'Model', 'Sampling', 'k', 'Value'])
    df.Model = df.Model.str.replace("mistral", "Mistral-7B").str.replace("mixtral", "Mixtral-8x22B").str.replace("gpt-3.5-turbo-0125", "GPT-3.5-Turbo")
    return df.set_index(['Schema', 'Entity Type', 'Metric', 'Model', 'Sampling', 'k']).unstack(['Schema', 'Entity Type', 'Metric'])

results_to_dataframe(results)


Value                                         \
Schema                      strict                                          
Entity Type                overall                                          
Metric                    f1_macro  f1_micro precision_macro recall_macro   
Model        Sampling k                                                     
llama3.1     rand     0   0.738178  0.740651        0.737136     0.739706   
                      5   0.735262  0.736457        0.712722     0.762472   
                      15  0.735549  0.737186        0.725281     0.747987   
                      25  0.690214  0.690983        0.705625     0.678611   
                      35  0.631078  0.631993        0.703716     0.576303   
             tfidf    5   0.721007  0.722298        0.696854     0.750226   
                      15  0.701056  0.702488        0.717145     0.687447   
                      25  0.688920  0.690323        0.711523     0.669867   
                      35  0.673498  0.676286        0.717500     0.635400   
llama3.1:70b rand     0   0.725325  0.726048        0.684567     0.777178   
                      5   0.742734  0.743183        0.715374     0.777292   
                      15  0.755722  0.756609        0.723942     0.794308   
                      25  0.752399  0.753160        0.720541     0.792085   
                      35  0.756981  0.757743        0.723814     0.798909   
             tfidf    5   0.728927  0.730738        0.710920     0.751808   
                      15  0.755710  0.757493        0.719550     0.797898   
                      25  0.763783  0.765629        0.728856     0.804554   
                      35  0.743490  0.745500        0.708855     0.784794   

                                                                           \
Schema                                                                      
Entity Type                 Artist                                          
Metric                          f1 precision    recall  missed   spurious   
Model        Sampling k                                                     
llama3.1     rand     0   0.771880  0.777775  0.766327   64.50  59.250000   
                      5   0.771534  0.775654  0.767728   60.50  57.000000   
                      15  0.765952  0.776771  0.755574   65.25  55.750000   
                      25  0.711690  0.761467  0.668204   96.75  54.000000   
                      35  0.663816  0.782045  0.576838  130.50  39.000000   
             tfidf    5   0.761278  0.765257  0.757735   64.75  61.500000   
                      15  0.723091  0.760302  0.689777   89.50  57.250000   
                      25  0.719175  0.771158  0.673844   94.50  50.500000   
                      35  0.722350  0.784650  0.669429   98.75  47.500000   
llama3.1:70b rand     0   0.771081  0.763302  0.779171   46.75  54.000000   
                      5   0.766981  0.775107  0.759295   55.50  48.250000   
                      15  0.779634  0.776669  0.782768   51.00  53.500000   
                      25  0.783828  0.786122  0.781634   51.75  49.750000   
                      35  0.792707  0.794904  0.790730   53.00  51.000000   
             tfidf    5   0.768830  0.778778  0.759763   59.00  50.666667   
                      15  0.789339  0.775793  0.803453   42.00  55.000000   
                      25  0.796731  0.785085  0.808730   41.50  52.500000   
                      35  0.787449  0.779464  0.795642   45.50  53.000000   

                                     ...                               \
Schema                               ...  ent_type                      
Entity Type                          ...    Artist                      
Metric                    incorrect  ...    recall  missed   spurious   
Model        Sampling k              ...                                
llama3.1     rand     0   16.500000  ...  0.791258   64.50  59.250000   
                      5   19.750000  ...  0.792

In [45]:
results

{'strict': {'overall': {'f1_macro': {'llama3.1': {'rand': {0: [0.7542972816657028,
       0.746309369630175,
       0.7223565048704834,
       0.7297473433358461],
      5: [0.7547052053824364,
       0.7200503978322028,
       0.7431342298363575,
       0.7231579041320249],
      15: [0.727951126915688,
       0.7385512994468219,
       0.7351003756312346,
       0.7405943604165084],
      25: [0.6992488216810683,
       0.7087946771304274,
       0.6718512647777557,
       0.6809617658651952],
      35: [0.6212442666343376,
       0.6221243153131697,
       0.6363492383448037,
       0.6445937158879935]},
     'tfidf': {5: [0.7148850237769526,
       0.7166666666666667,
       0.699407114624506,
       0.7530699088145897],
      15: [0.7095959595959596,
       0.7054261121795882,
       0.6846145288686172,
       0.704587869362364],
      25: [0.6797750921248642,
       0.700946369093919,
       0.6746543526968687,
       0.7003048311901224],
      35: [0.6910242982823629,
       0.6

# Precision

In [46]:
_results = results_to_dataframe(results, "precision", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)


TypeError: results_to_dataframe() takes from 1 to 2 positional arguments but 3 were given

# Recall

In [ ]:
_results = results_to_dataframe(results, "recall", "mean").reset_index()
_results[(_results.Schema == "strict") & (_results.k == 0)].drop(["Schema", "k"], axis=1)


,Entity Type,Model,Value
0,Artist,llama3.1,0.756232
5,Artist,llama3.1-70b,0.779171
6,WoA,llama3.1,0.716194
11,WoA,llama3.1-70b,0.775184


# F1

In [ ]:
results_to_dataframe(results, "f1_macro", "mean")


Value
Schema   Entity Type Model        k           
strict   overall     llama3.1     0   0.736789
                                  5   0.736789
                                  15  0.736789
                                  25  0.736789
                                  35  0.736789
                     llama3.1-70b 0   0.725325
exact    overall     llama3.1     0   0.751727
                                  5   0.751727
                                  15  0.751727
                                  25  0.751727
                                  35  0.751727
                     llama3.1-70b 0   0.758958
ent_type overall     llama3.1     0   0.779673
                                  5   0.779673
                                  15  0.779673
                                  25  0.779673
                                  35  0.779673
                     llama3.1-70b 0   0.768855

In [ ]:
results_to_dataframe(results, "f1", "mean")


Value
Schema   Entity Type Model        k          
strict   Artist      llama3.1-70b 0  0.771081
                     llama3.1-8b  0  0.763753
         WoA         llama3.1-70b 0  0.679570
                     llama3.1-8b  0  0.709825
exact    Artist      llama3.1-70b 0  0.833947
                     llama3.1-8b  0  0.784851
         WoA         llama3.1-70b 0  0.683969
                     llama3.1-8b  0  0.718603
ent_type Artist      llama3.1-70b 0  0.793239
                     llama3.1-8b  0  0.788424
         WoA         llama3.1-70b 0  0.744471
                     llama3.1-8b  0  0.770921